I did not do a proper 12 year spin up for methane in geos chem, so going to scale the geos chem simulations to aircore. 

Josh L said: " Yes, it is possible to use aircore profiles as the priors GGG needs, and in fact that’s what we do when we are calculating the in situ scale factor. The tricky part is that we only have about 20 aircore profiles at Lamont between 2012 and 2018, so filling in the intervening periods between profiles would be tricky.
 
Scaling the profiles should work. I wouldn’t worry too much about the variability. I think if you try to correct that, you could pretty easily get into a situation with overfitting the sparse aircore data. I wouldn’t scale the whole profile though; notice that the stratosphere matches up pretty well between GC and aircore.
 
What I’d probably do is compare the GC output below the tropopause with all the aircore profiles and try to find a single scaling factor that brings the GC troposphere into reasonably unbiased agreement with the aircores. To apply that, I’d multiply all GC CH4 below the tropopause by that scaling factor and leave everything above 380 K potential temperature alone. Then for each GC grid cell, I’d connect the top level below the tropopause to the bottom level above 380 K PT by interpolating the CH4 w.r.t. potential temperature.
 
This is similar to what we do in the priors, it works because levels above 380 K are “pure” stratosphere, so they follow large scale convective motion and are easier for models to capture. Interpolating w.r.t. PT works as a good way to connect the troposphere and stratosphere because it’s a good tracer for mixing between the strat and trop."

In [1]:
import csv
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import glob
import sys
import pytz
from matplotlib.cm import get_cmap
import cartopy.crs as ccrs, bokeh
import cartopy.feature as cfeature
import bokeh
print(bokeh.__version__)
import bokeh.io
import bokeh.plotting
from bokeh.plotting import figure
bokeh.io.output_notebook()

2.3.3


Loading BokehJS ...

Load aircore files

Lamont lat,lon : 36.69109, -97.41193


Looking for 2016, 2017, 2018, 2019, 2020. 

Here is what I found for areas near Lamont (some are closeby, like Enid): 
```
/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20140917/launch2/AirCoreGMD003_201409171757_v20200103.txt
```


2016-2018:
```
/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20161018/launch1/AirCoreGMD007_201610181722_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20161018/launch2/AirCoreGMD008_201610181722_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20161019/launch1/AirCoreGMD007_201610191712_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20161019/launch2/AirCoreGMD008_201610191712_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20170412/launch3/AirCoreGMD008_201704121649_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20170411/launch2/AirCoreGMD008_201704111728_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20170411/launch3/AirCoreGMD007_201704111728_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20180723/launch1/AirCoreGMD007_201807231659_v20201223.txt 

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20180723/launch2/AirCoreGMD009_201807231659_v20201223.txt 

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20180725/launch1/AirCoreGMD008_201807251716_v20200103.txt 

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20180725/launch2/AirCoreGMD009_201807251716_v20200103.txt 

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20180725/launch3/AirCoreGMD007_201807251658_v20200103.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20180725/launch4/AirCoreGMD010_201807251658_v20200103.txt

```

Since there aren't Lamont profiles for 2019-2020, you should just use any other locations since all you want is the background anyway. Here is what I am choosing: 

```
/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20190618/launch1/AirCoreGMD008_201906180800_v20200619.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20190514/launch1/AirCoreGMD007_201905141700_v20201209.txt

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20190514/launch2/AirCoreGMD010_201905141700_v20200103.txt 

/oco2-data/tccon/analysis/jlaugh/Notebooks/TCCON/GGG2020-AICF/Data/aircores/v20201223/level2/Flight_20200305/launch2/AirCoreGMD007_202003051800_v20201211.txt
```

In [2]:
path16 = "/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCore*2016*.txt"
inputs16 = sorted(glob.glob(path16))

In [3]:
inputs16

['/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCoreGMD007_201610181722_v20200103.txt',
 '/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCoreGMD007_201610191712_v20200103.txt',
 '/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCoreGMD008_201610181722_v20200103.txt',
 '/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCoreGMD008_201610191712_v20200103.txt']

In [4]:
df_2016 = pd.concat((pd.read_csv(f) for f in inputs16), ignore_index=True)

Double check it worked to concatenate all of the files together

In [5]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')

p.circle(df_2016[' TH(K)'].values,
         df_2016[' CH4(ppb)'].values, size=9, 
         color=colors[10],legend_label='ac all')

p.circle(pd.read_csv(inputs16[0])[' TH(K)'].values,
         pd.read_csv(inputs16[0])[' CH4(ppb)'].values, size=5, 
         color=colors[6],legend_label='ac 1')

p.circle(pd.read_csv(inputs16[1])[' TH(K)'].values,
         pd.read_csv(inputs16[1])[' CH4(ppb)'].values, size=5, 
         color=colors[4],legend_label='ac 2')

p.circle(pd.read_csv(inputs16[2])[' TH(K)'].values,
         pd.read_csv(inputs16[2])[' CH4(ppb)'].values, size=5, 
         color=colors[3],legend_label='ac 3')

p.circle(pd.read_csv(inputs16[3])[' TH(K)'].values,
         pd.read_csv(inputs16[3])[' CH4(ppb)'].values, size=5, 
         color=colors[1],legend_label='ac 4')

p.xaxis.axis_label = "theta (K)"
p.yaxis.axis_label = "CH4 (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

How many files are in each year

In [6]:
path17 = "/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCore*_2017*_v20200103.txt"
path18 = "/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCore*_2018*_v20200103.txt"
path19 = "/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCore*_2019*.txt"
path20 = "/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AirCore*_2020*.txt"



print(len(sorted(glob.glob(path16))))
print(len(sorted(glob.glob(path17))))
print(len(sorted(glob.glob(path18))))
print(len(sorted(glob.glob(path19))))
print(len(sorted(glob.glob(path20))))


4
3
4
2
1


In [7]:
#2016
lennums_16 = [len(pd.read_csv(f)) for f in sorted(glob.glob(path16))]
acnum_16 = np.concatenate((np.zeros(lennums_16[0]),
                           np.ones(lennums_16[1]),
                           np.ones(lennums_16[2])*2,
                           np.ones(lennums_16[3])*3)).astype(int)
df_2016.insert(0,"campaign_num",acnum_16,True)

#2017
df_2017 = pd.concat((pd.read_csv(f) for f in sorted(glob.glob(path17))), ignore_index=True)
lennums_17 = [len(pd.read_csv(f)) for f in sorted(glob.glob(path17))]
acnum_17 = np.concatenate((np.zeros(lennums_17[0]),
                           np.ones(lennums_17[1]),
                           np.ones(lennums_17[2])*2)).astype(int)
df_2017.insert(0,"campaign_num",acnum_17,True)

#2018
df_2018 = pd.concat((pd.read_csv(f) for f in sorted(glob.glob(path18))), ignore_index=True)
lennums_18 = [len(pd.read_csv(f)) for f in sorted(glob.glob(path18))]
acnum_18 = np.concatenate((np.zeros(lennums_18[0]),
                           np.ones(lennums_18[1]),
                           np.ones(lennums_18[2])*2,
                           np.ones(lennums_18[3])*3)).astype(int)
df_2018.insert(0,"campaign_num",acnum_18,True)

#2019
df_2019 = pd.concat((pd.read_csv(f) for f in sorted(glob.glob(path19))), ignore_index=True)
lennums_19 = [len(pd.read_csv(f)) for f in sorted(glob.glob(path19))]
acnum_19 = np.concatenate((np.zeros(lennums_19[0]),np.ones(lennums_19[1]))).astype(int)
df_2019.insert(0,"campaign_num",acnum_19,True)

#2020
df_2020 = pd.concat((pd.read_csv(f) for f in sorted(glob.glob(path20))), ignore_index=True)
lennums_20 = [len(pd.read_csv(f)) for f in sorted(glob.glob(path20))]
acnum_20 = (np.zeros(lennums_20[0])).astype(int)
df_2020.insert(0,"campaign_num",acnum_20,True)


Now get the original aircore file that you compared with before

In [8]:
def converttodf(inputs):
    fieldnames = []
    for filename in inputs:
      with open(filename, "r", newline="") as f_in:
        reader = csv.reader(f_in)
        headers = next(reader)
        for h in headers:
          if h not in fieldnames:
            fieldnames.append(h)


    # # # Then copy the data
    with open("out.csv", "w", newline="") as f_out:   # Comment 2 below
      writer = csv.DictWriter(f_out, fieldnames=fieldnames)
      writer.writeheader()
      for filename in inputs:
        with open(filename, "r", newline="") as f_in:
          reader = csv.DictReader(f_in)  # Uses the field names in this file
          for line in reader:
            writer.writerow(line)

    df = pd.read_csv("out.csv")

    # insert NaN for -9999 values (be careful of lon values that are negative) 
    df.loc[:,:] < -99998 
    df_raw = df.where(~(df.loc[:,:] < -99998), other= np.NaN)
    
    complete_time_all_campaigns = []
    # insert completetime ..........................................
    for i,x in enumerate(inputs):
        
#         dateyr = x.split('AC_GMD007_')[1].rsplit('_')[0][0:4]
#         datemn = x.split('AC_GMD007_')[1].rsplit('_')[0][4:6]
#         datedy = x.split('AC_GMD007_')[1].rsplit('_')[0][6:8]
        dateyr = x.rsplit("_",2)[1].rsplit('_')[0][0:4]
        datemn = x.rsplit("_",2)[1].rsplit('_')[0][4:6]
        datedy = x.rsplit("_",2)[1].rsplit('_')[0][6:8]
        

        file_df = pd.read_csv(inputs[i])
        file_time_list = file_df['Mid_UTC'].tolist()
        
        
        # add completetime
        utc = pytz.UTC
        file_name_dtdelta = datetime.datetime(int(dateyr), int(datemn), int(datedy), 0, 0, 0)
        file_name_dtdelta = utc.localize(file_name_dtdelta)
        total_time = [datetime.timedelta(0,x)+file_name_dtdelta for x in file_time_list]
        complete_time_all_campaigns.append(total_time)

    complete_time_all_campaigns = np.concatenate(complete_time_all_campaigns).ravel()
    df_raw['completetime'] = complete_time_all_campaigns
    
    return df_raw

In [9]:
path = "/Users/arianatribby/git/oklahoma_propane/data/tccon_aircore/AC_GMD010_201807251658_R0.ict.csv"
inputs = sorted(glob.glob(path))



ac_orig_df = converttodf(inputs)

Now, test only for 2016 by scaling with GC sim of CH4. This is what you should be doing on AWS. Note that not all profiles will be exactly in Lamont, but might be nearby. I'm not changing this lat/lon since it is course anyway. 36.69109, -97.41193

In [10]:
xs_gcsim = xr.open_dataset("/Users/arianatribby/git/oklahoma_propane/code/processing/select_vars_20160601_20160605.nc4")

In [11]:
max(df_2017[' TH(K)'].values)

604.265571623701

In [12]:
df_gcsim_ch4 = xs_gcsim[['Met_THETA','SpeciesConc_CH4','Met_PMID']].to_dataframe().reset_index()
df_gcsim_ch4lamont = df_gcsim_ch4.loc[(df_gcsim_ch4['lon'] > -100) & (df_gcsim_ch4['lon'] < -90) &
                                     (df_gcsim_ch4['lat'] > 30) & (df_gcsim_ch4['lat'] < 38) & 
                                     (df_gcsim_ch4['Met_THETA'] < 605)]

gcsim_nearlamont = df_gcsim_ch4.loc[(df_gcsim_ch4['lon'] < -90.) & (df_gcsim_ch4['lon'] > -100) & 
                                      (df_gcsim_ch4['lat'] < 38) & (df_gcsim_ch4['lat'] > 30)]

df_gcsim_ch4lamont_ts = df_gcsim_ch4.loc[(df_gcsim_ch4['lon'] > -100) & (df_gcsim_ch4['lon'] < -90) &
                                     (df_gcsim_ch4['lat'] > 30) & (df_gcsim_ch4['lat'] < 38) & 
                                     (df_gcsim_ch4['Met_THETA'] < 605) & 
                                    (df_gcsim_ch4['time'] == '2016-06-01 01:30:00')]

Lets see what theta looks like for both to see what to interpolate to. 

In [13]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')

p.circle(df_gcsim_ch4lamont['Met_THETA'].values,
         df_gcsim_ch4lamont['SpeciesConc_CH4'].values*1e9, size=5, 
         color=colors[3],legend_label='gc')

p.circle(df_2016[' TH(K)'].values,
         df_2016[' CH4(ppb)'].values, size=5, 
         color=colors[1],legend_label='ac')

p.circle(ac_orig_df['THETA'].values,
         ac_orig_df['CH4'].values, size=5, color=colors[4], legend_label='ac 2')

p.xaxis.axis_label = "theta (K)"
p.yaxis.axis_label = "CH4 (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

In [14]:
df_2016.columns

Index(['campaign_num', 'DateTime(yyyy-mm-dd HH:MM:SS)',
       ' SecondsFromMidnight(s)', ' Pressure(mb)', ' Lat(N)', ' Lon(E)',
       ' Alt(m)', ' CO2(ppm)', ' co2_err(ppm)', ' CH4(ppb)', ' ch4_err',
       ' CO(ppb)', ' co_err(ppb)', ' T(C)', ' TH(K)', ' RH(%)', ' AirCoreT(C)',
       ' alt_err_co2-(m)', ' alt_err_co2+(m)', ' alt_err_ch4-(m)',
       ' alt_err_ch4+(m)', ' alt_err_co-(m)', ' alt_err_co+(m)'],
      dtype='object')

In [15]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')


p.circle(df_gcsim_ch4lamont_ts['Met_PMID'].values,
         df_gcsim_ch4lamont_ts['SpeciesConc_CH4'].values*1e9, size=5, 
         color=colors[3],legend_label='gc')


# p.circle(df_gcsim_ch4lamont['Met_PMID'].values,
#          df_gcsim_ch4lamont['SpeciesConc_CH4'].values*1e9, size=5, 
#          color=colors[3],legend_label='gc')

p.circle(df_2016[' Pressure(mb)'].values,
         df_2016[' CH4(ppb)'].values, size=5, 
         color=colors[1],legend_label='ac')

p.circle(ac_orig_df['P'].values,
         ac_orig_df['CH4'].values, size=5, color=colors[4], legend_label='ac 2')

p.xaxis.axis_label = "pressure"
p.yaxis.axis_label = "CH4 (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "bottom_right"
# p.output_backend = "svg"
bokeh.io.show(p)

Above 380 is stratosphere. Only want to use scalar for troposphere. This also means removing the boundary layer values (0km = 1000mb pressure, 1km is about 900 mb pressure). Boundary layer stretches to about 1km. 

In [16]:
df_gcsim_ch4lamont.columns

Index(['time', 'lev', 'lat', 'lon', 'Met_THETA', 'SpeciesConc_CH4',
       'Met_PMID'],
      dtype='object')

In [17]:
gcsim_lamont_trop = df_gcsim_ch4lamont.loc[(df_gcsim_ch4lamont['Met_THETA'] < 380) & 
                                           (df_gcsim_ch4lamont['Met_PMID'] < 900)]
ac_trop_16 = df_2016.loc[(df_2016[' TH(K)'] < 380) & (df_2016[' Alt(m)'] > 1000)]

In [18]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')

p.circle(gcsim_lamont_trop['Met_THETA'].values,
         gcsim_lamont_trop['SpeciesConc_CH4'].values*1e9, size=5, 
         color=colors[3],legend_label='gc')

p.circle(ac_trop_16[' TH(K)'].values,
         ac_trop_16[' CH4(ppb)'].values, size=5, 
         color=colors[1],legend_label='ac')

p.xaxis.axis_label = "theta (K)"
p.yaxis.axis_label = "CH4 (ppb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

Get an average aircore potential temperature to serve as the interpolation coordinate. 

In [19]:
ac_trop_16.columns

Index(['campaign_num', 'DateTime(yyyy-mm-dd HH:MM:SS)',
       ' SecondsFromMidnight(s)', ' Pressure(mb)', ' Lat(N)', ' Lon(E)',
       ' Alt(m)', ' CO2(ppm)', ' co2_err(ppm)', ' CH4(ppb)', ' ch4_err',
       ' CO(ppb)', ' co_err(ppb)', ' T(C)', ' TH(K)', ' RH(%)', ' AirCoreT(C)',
       ' alt_err_co2-(m)', ' alt_err_co2+(m)', ' alt_err_ch4-(m)',
       ' alt_err_ch4+(m)', ' alt_err_co-(m)', ' alt_err_co+(m)'],
      dtype='object')

The pottemp from the aircores are not the same length. What do you want to do. How do you want to interp ch4 to aircore. maybe first interp aircore to the longest or cut all the aircore to the shortest length. 

In [20]:
print(len(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)']))
print(len(ac_trop_16.loc[ac_trop_16['campaign_num'] == 1][' TH(K)']))
print(len(ac_trop_16.loc[ac_trop_16['campaign_num'] == 2][' TH(K)']))
print(len(ac_trop_16.loc[ac_trop_16['campaign_num'] == 3][' TH(K)']))

470
463
469
469


In [21]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')


p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 2][' TH(K)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 2][' Pressure(mb)'], size=5, 
         color=colors[4],legend_label='c 2')

p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' Pressure(mb)'], size=5, 
         color=colors[2],legend_label='c 0')

p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 1][' TH(K)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 1][' Pressure(mb)'], size=5, 
         color=colors[3],legend_label='c 1')

p.xaxis.axis_label = "theta (K)"
p.yaxis.axis_label = "p (mb)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

Generally what is value of theta for boundary layer (1km or less), bc theta will not be a good coordinate there. 

In [22]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')

p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 2][' Pressure(mb)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 2][' Alt(m)'], size=5, 
         color=colors[4],legend_label='c 2')

p.xaxis.axis_label = "p (mb)"
p.yaxis.axis_label = "alt (m)"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

In [23]:
gc_interp_16_0 = np.interp(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'].values,
         gcsim_lamont_trop['Met_THETA'].values,
         gcsim_lamont_trop['SpeciesConc_CH4'].values*1e9)

In [24]:
scale_16_0 = (gc_interp_16_0 - ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'].values)/2

In [25]:
testconv = np.convolve(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' CH4(ppb)'],gc_interp_16_0)

In [26]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w


In [27]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')


p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'],
         gc_interp_16_0, size=5, 
         color=colors[4],legend_label='gc interppd')

p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' CH4(ppb)'], size=5, 
         color=colors[2],legend_label='ac 0')

# p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 1][' TH(K)'],
#          moving_average(gc_interp_16_0,int(len(gc_interp_16_0)*.50)), size=5, 
#          color=colors[3],legend_label='gc mova')

p.xaxis.axis_label = "theta (K)"
p.yaxis.axis_label = "ch4"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

In [28]:
fh = 300
fw = 300
colors = bokeh.palettes.d3['Category20'][20]


p = bokeh.plotting.figure(frame_height=fh, frame_width=fw, title='')


p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' Pressure(mb)'],
         gc_interp_16_0, size=5, 
         color=colors[4],legend_label='gc interppd')

p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' Pressure(mb)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' CH4(ppb)'], size=5, 
         color=colors[2],legend_label='ac 0')

# p.circle(ac_trop_16.loc[ac_trop_16['campaign_num'] == 1][' TH(K)'],
#          moving_average(gc_interp_16_0,int(len(gc_interp_16_0)*.50)), size=5, 
#          color=colors[3],legend_label='gc mova')

p.xaxis.axis_label = "theta (K)"
p.yaxis.axis_label = "ch4"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.label_text_font_size = '14pt'
p.legend.location = "top_right"
# p.output_backend = "svg"
bokeh.io.show(p)

In [29]:
np.array([ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'].values,
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 1][' TH(K)'].values,
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 2][' TH(K)'].values,
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 3][' TH(K)'].values])

<ipython-input-29-7439af7f666f>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'].values,


array([array([379.84148371, 379.78190507, 379.12861268, 378.36906808,
              377.39229683, 376.62602278, 376.21820059, 374.12890925,
              372.88297498, 372.39782824, 372.10055992, 371.66744089,
              370.89698781, 370.37682951, 370.35739846, 369.57786724,
              368.29785314, 367.39507641, 366.85413692, 366.80404593,
              366.09367265, 364.99728243, 364.461342  , 364.13910904,
              363.6551984 , 363.55192079, 363.23089503, 362.78664059,
              361.44860792, 360.4608709 , 359.89415029, 359.06885484,
              358.65763342, 358.52872758, 358.24197018, 357.30332772,
              356.53837091, 356.25638336, 356.16230325, 356.13620409,
              355.5639082 , 354.60528477, 352.80319379, 351.57793892,
              350.76540781, 350.21224239, 349.61567651, 348.83932991,
              348.25727999, 347.64082245, 347.25431089, 346.73768172,
              346.30532403, 346.00718437, 345.74676422, 345.70363733,
              345.47

In [30]:
np.mean([ac_trop_16.loc[ac_trop_16['campaign_num'] == 0][' TH(K)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 1][' TH(K)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 2][' TH(K)'],
         ac_trop_16.loc[ac_trop_16['campaign_num'] == 3][' TH(K)']],1)

/Users/arianatribby/opt/anaconda3/envs/oklahoma_propane_env/lib/python3.8/site-packages/numpy/core/_methods.py:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


AxisError: axis 1 is out of bounds for array of dimension 1